In [1]:
import matplotlib
matplotlib.use("TkAgg") #backend
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg,NavigationToolbar2TkAgg
from matplotlib.figure import Figure
import matplotlib.animation as animation
from matplotlib import style
from matplotlib import pyplot as plt

In [2]:
import tkinter as tk
from tkinter import ttk #for good looking buttons

In [3]:
import urllib
import json
import pandas as pd
import numpy as np

In [4]:
LARGE_FONT= ("Verdana",12)
NORM_FONT= ("Verdana",10)
SMALL_FONT= ("Verdana",8)
style.use("ggplot")

In [5]:
# f = Figure(figsize=(10,4),dpi=100)
f = Figure()
a = f.add_subplot(111)
# default values
exchange = "BTC-e" # starting exchange
DatCounter = 9000 # when we are updating any thing
programName = "btce"


In [6]:
def changeExcahnge(toWhat,pn):
    global exchange
    global DatCounter
    global programName
    toWhat = exchange
    programName = pn
    DatCounter = 9000 # request any update then it must occure immediately

In [7]:
def popupmsg(msg):
    popup = tk.Tk()
    #popup.wm_title("!")
    label = ttk.Label(popup,text=msg,font=NORM_FONT)
    label.pack(side="top",fill="x",pady=10)
    B1 = ttk.Button(popup, text="Okay",command=popup.destroy)
    B1.pack()
    popup.mainloop()

In [8]:
def animate(i):
    pullData = open("sampleData.txt","r").read()
    dataList = pullData.split('\n')
    xList = []
    yList = []
    zList = []
    for eachLine in dataList:
        if len(eachLine)>1:
            x,y,z = eachLine.split(',')
            xList.append(int(x))
            yList.append(y)
            zList.append(z)
    a.clear()
    a.plot(xList,yList,"#00A3E0",label="buys")
    a.plot(xList,zList,"#183A54",label="sells")
    a.legend(bbox_to_anchor=(0,1.02,1,.102),loc=3,
            ncol=2,borderaxespad=0)
    title = "BTC-e BTCUSD Price"
    a.set_title(title)

In [22]:
class seaofBTCapp(tk.Tk): # inherit the attributes of another class 
    
    def __init__(self, *args,**kwargs): 
        # initialize with your class
        # *args- short form of arguments, you can pass any number of variables
        # **kwargs- keyword arguments, passing through dictionaries 
        
        tk.Tk.__init__(self, *args, **kwargs)
        
#         tk.Tk.iconbitmap(self,default="ball.xbm")
        tk.Tk.wm_title(self,"Sea of BTC client")
        container = tk.Frame(self)
        # contain everything of our tkinter app ,or edge of window
        container.pack(side="top", fill="both", expand= True)
        # pack and grid are two option to arrange
        
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        # defining the menu bar
        menubar = tk.Menu(container)
        
        #defining the File menu bar
        filemenu = tk.Menu(menubar,tearoff=0)
        filemenu.add_command(label="Save settings", command=lambda:popupmsg("Not supported just yet!!"))
        filemenu.add_separator()
        filemenu.add_command(label="Exit",command=quit)
        menubar.add_cascade(label="File",menu=filemenu)
        
        # defining the Exchange menu bar
        exchangeChoice = tk.Menu(menubar,tearoff=1)
        exchangeChoice.add_command(label="BTC-e",
                                  command=lambda:changeExcahnge("BTC-e","btce"))#(displayName,programName)
        exchangeChoice.add_command(label="Bitfinex",
                                  command=lambda:changeExcahnge("Bitfinex","bitfinex"))
        exchangeChoice.add_command(label="Bitstamp",
                                  command=lambda:changeExcahnge("Bitstamp","bitstamp"))
        exchangeChoice.add_command(label="Huobi",
                                  command=lambda:changeExcahnge("Huobi","huobi"))
        menubar.add_cascade(label="Exchange",menu=exchangeChoice)
        
        # defining the consider time menu bar
        dataTF = tk.Menu(menubar,tearoff=1)
        dataTF.add_command(label="Tick",
                          command=lambda:changeTimeFrame("tick"))
        dataTF.add_command(label="1 Day",
                          command=lambda:changeTimeFrame("1d"))
        dataTF.add_command(label="3 Day",
                          command=lambda:changeTimeFrame("3d"))
        dataTF.add_command(label="1 Week",
                          command=lambda:changeTimeFrame("7d"))
        menubar.add_cascade(label="Data Time Frame",menu=dataTF)
        
        # defining the open high low close interval as OHLCI menu bar
        OHLCI = tk.Menu(menubar,tearoff=1)
        OHLCI.add_command(label="Tick",
                          command=lambda:changeTimeFrame("tick"))
        OHLCI.add_command(label="1 minute",
                          command=lambda:changeSampleSize("1Min",0.0005))
        OHLCI.add_command(label="5 minute",
                          command=lambda:changeSampleSize("5Min",0.003))
        OHLCI.add_command(label="15 minute",
                          command=lambda:changeSampleSize("15Min",0.008))
        OHLCI.add_command(label="30 minute",
                          command=lambda:changeSampleSize("30Min",0.016))
        OHLCI.add_command(label="1 hour",
                          command=lambda:changeSampleSize("1H",0.032))
        OHLCI.add_command(label="3 hour",
                          command=lambda:changeSampleSize("3H",0.096))
        menubar.add_cascade(label="OHLC Interval",menu=OHLCI)
        
        tk.Tk.config(self,menu=menubar)
        
        self.frames ={}
        
        for F in (StartPage,BTCe_Page):#(StartPage,PageOne,PageTwo,PageThree):

            frame = F(container,self)

            self.frames[F] = frame

            frame.grid(row=0,column=0,sticky="nsew")
        
        self.show_frame(StartPage)
        
    def show_frame(self,cont):
        
        frame = self.frames[cont]
        frame.tkraise()

In [20]:
class StartPage(tk.Frame):
    def __init__(self,parent,controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text=("""ALPHA Bitcoin trading application
        use at your own risk. There is no 
        promise of WARRANTY"""), font= LARGE_FONT)
        label.pack(pady=10,padx=10)
        
        button1 = ttk.Button(self, text="Agree",
                           command=lambda: controller.show_frame(BTCe_Page))
        button1.pack()
        
        button2 = ttk.Button(self, text="Disagree",
                           command=quit)
        button2.pack()
        

In [11]:
class PageOne(tk.Frame):
    def __init__(self,parent,controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Page One", font= LARGE_FONT)
        label.pack(pady=10,padx=10)
        
        button1 = ttk.Button(self, text="Back to home",
                           command=lambda: controller.show_frame(StartPage))
        button1.pack()
        

In [12]:
class BTCe_Page(tk.Frame):
    def __init__(self,parent,controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Graph Page", font= LARGE_FONT)
        label.pack(pady=10,padx=10)
        
        button1 = ttk.Button(self, text="Back to home",
                           command=lambda: controller.show_frame(StartPage))
        button1.pack()
        
        
        canvas = FigureCanvasTkAgg(f,self)
        canvas.show()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand= True)
        
        tollbar = NavigationToolbar2TkAgg(canvas,self)
        tollbar.update()
        canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand= True)

In [16]:
app = seaofBTCapp()
app.geometry("1280x720")
ani = animation.FuncAnimation(f,animate,interval=1000)
# app.mainloop()

In [14]:
# app.mainloop()